In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import cvxpy as cp
from tqdm import tqdm
from pathlib import Path

from src.shortest_paths_gt import get_graph_props
from src.load_data import (
    read_metadata_networks_tntp,
    read_graph_transport_networks_tntp,
    read_traffic_mat_transport_networks_tntp,
) 
from src.models import SDModel, BeckmannModel, TwostageModel
from src.algs import subgd, ustm, frank_wolfe, cyclic
from src.cvxpy_solvers import get_max_traffic_mat_mul
from src.commons import Correspondences

import matplotlib.pyplot as plt

plt.rcParams.update({"font.size": 14})
%config InlineBackend.figure_format = 'retina'

%matplotlib inline
networks_path = Path("./TransportationNetworks")

folder = "SiouxFalls"
net_name = "SiouxFalls_net"
traffic_mat_name = "SiouxFalls_trips"

net_file = networks_path / folder / f"{net_name}.tntp"
traffic_mat_file = networks_path / folder / f"{traffic_mat_name}.tntp"
graph, metadata = read_graph_transport_networks_tntp(net_file)
correspondences = read_traffic_mat_transport_networks_tntp(traffic_mat_file, metadata)
n = graph.number_of_nodes()

print(f"{graph.number_of_edges()=}, {graph.number_of_nodes()=}")
beckmann_model = BeckmannModel(graph, correspondences)

eps = 1e-4
mean_bw = beckmann_model.graph.ep.capacities.a.mean()
mean_cost = beckmann_model.graph.ep.free_flow_times.a.mean()


eps_abs = eps * mean_cost * mean_bw * graph.number_of_edges()

eps_cons_abs = eps * mean_bw 

# sum of capacity violation <= eps * average link capacity
print(eps_abs, eps_cons_abs)

# Compare with Frank-Wolfe method
#times_e_fw, flows_e_fw, logs, optimal = frank_wolfe(beckmann_model, eps_abs)
times_e_fw, flows_e_fw, _, _ = frank_wolfe(beckmann_model, eps_abs)


In [14]:
def S(e,A,y):
    product = -np.dot(A.T,y)[e]
#     print(product,e)
    max_ = np.max(product)
    #print(max_.shape)
    i_star = np.argwhere(product==max_)[0][0]
    return np.array([max_, i_star])
def gradient_sigma_star_e(s,e,t,f,rho,mu):
    fe = f[e]
    te = t[e]
#     print(f"{s,e,te,fe,rho,mu=}")
    п = max(s, te)
    result = fe*(((п-te)/(te*rho))**mu)  * (s - te > 0)
#     s,e,te,fe,rho,mu=(-0.0, 0, 6.0, 25900.201171875, 0.15000000596046448, 0.25)
    return result
def calculate_M(e,A,y):
    M = np.zeros_like(y)
    i_star = S(e,A,y)[1]
    ## - A_me * delta_i*n = M_mn
    for m in range(M.shape[0]):
        for n in range(M.shape[1]):
            if n == i_star:
                delta = 1
            else:
                delta = 0
            M[m][n] = - A[m][e] * delta
    return M

def gradient_fy(y,A,t,f,rho,mu,L):
    sum_ = 0
    for e in range(A.shape[0]):
        s = S(e,A,y)[0]
        sigma_ = gradient_sigma_star_e(s,e,t,f,rho,mu)
        Me = calculate_M(e,A,y)
        product = sigma_ * Me
        sum_ += product
#         print(f"{e, sum_ = }")
    sum_ -= L
    return sum_

def grady(y):
    return gradient_fy(y,incidence_matrix,fft,caps,rho,mu,L)
# def compute_flow_values(optimized_y, incidence_matrix):
#     flow_values = np.dot(incidence_matrix.T, optimized_y).sum(axis=1)
#     return flow_values
def compute_flow_values(optimized_y, incidence_matrix):
    z = np.min(incidence_matrix.T @ optimized_y, axis=1)
    amin = np.argmin(incidence_matrix.T @ optimized_y, axis=1)
    
    flow_values = np.zeros((incidence_matrix.T.shape))
#     amin_ind_tuples = np.vstack((np.arange(flow_values.shape[0]), amin)).T
    flow_values[np.arange(flow_values.shape[0]), amin] = beckmann_model.tau_inv(np.maximum(fft, -z))
    
#     flow_values[*amin_ind_tuples] = beckmann_model.tau_inv(np.maximum(fft, -z))
    assert np.all(np.isfinite(flow_values))
    return flow_values

def gradient_descent(y_init, learning_rate, num_iterations, fw_results):
    y = y_init
    flow_values_sum = np.zeros(incidence_matrix.T.shape)
    cons_log = []
    for i in range(num_iterations):
        flow_values = compute_flow_values(y, incidence_matrix)
        grad = incidence_matrix @ flow_values + L.T
        y += learning_rate * grad
        flow_values_sum += flow_values

        if not i % 1000:  # Update the plot with both FW and GD results
            plt.plot(fw_results, label="FW")  # Use orange color for FW results
            plt.plot(flow_values_sum.sum(axis=1) / (i + 1), label="GD")  # Use blue color for GD results
            plt.xlabel('Edge index')  # Change x-axis label to 'Edge index'
            plt.ylabel('Flow on edge')
            plt.legend(), plt.show()

        cons_log.append(np.linalg.norm(incidence_matrix @ flow_values_sum / (i + 1) + L.T))

    flow_values_avg = flow_values_sum / num_iterations
    return y, flow_values_avg, cons_log


In [15]:
# def softmin(x, beta):
#     """ Compute the smooth minimum of an array x with smoothing parameter beta """
# #     max_x = np.max(x)
# #     x_stable = x - max_x
# #     weights = np.exp(-beta * x_stable)
# #     return np.dot(weights, x) / np.sum(weights)
#     return -logsumexp(-beta * x) / beta
# #     return -logsumexp(-x)

# # # Function to compute sigma_star, which is the sum of squares of the elements
# # def sigma_star(t):
# #     """ Compute the sum of squares of the elements of the array t """
# #     return (t ** 2).sum()

def numerical_gradient(f, x, eps=1e-8):
    # Set up an array to store our gradient calculations.
    grad = np.zeros_like(x)
    
    # Let's go through each element in x to find the gradients.
    for i in range(len(x)):
        x_plus = x.copy()
        x_plus[i] += eps  # Nudge the element up a bit by eps.
        
        x_minus = x.copy()
        x_minus[i] -= eps  # Now nudge it down by eps and see what happens.
        
        # Calculate the gradient using the central difference formula.
        grad[i] = (f(x_plus) - f(x_minus)) / (2 * eps)
    
    return grad


def softmin(x, beta):
    """ Compute the smooth minimum of an array x with smoothing parameter beta """
    return -logsumexp(-beta * x) / beta


# Define the dual objective function using the softmin
def dual_objective(y_flattened, A, L, beta):
    """ The dual objective function to be minimized """
    y = y_flattened.reshape((nodes, nodes))
    ATy = A.T @ y
    
    # Apply softmin to each line of ATy
    softmin_values = np.array([softmin(ATy_row, beta) for ATy_row in ATy])
    
    # Calculate the value of the target function
    objective_value = np.sum(beckmann_model.sigma_star(-softmin_values)) - np.sum(y * L)
    
    if np.isnan(objective_value) or np.isinf(objective_value):
        print("NaN or Inf detected in objective function!")
        return np.finfo(float).max
    
    return objective_value

def dual_objective_grad(y_flattened, A, L, beta):
    """ Gradient of the target function of the dual problem """
    y = y_flattened.reshape((nodes, nodes))
    ATy = A.T @ y
    
    # Calculate softmin for each line of ATy
    softmin_values = np.array([softmin(ATy_row, beta) for ATy_row in ATy])
    
    # Calculate the derivative of softmin
    softmin_grad = np.exp(-beta * (ATy - softmin_values[:, np.newaxis]))
    softmin_grad /= np.sum(softmin_grad, axis=1, keepdims=True)
    
    # Calculate the sigma_star gradient numerically
    sigma_star_grad = numerical_gradient(lambda x: np.sum(beckmann_model.sigma_star(x)), -softmin_values)
    
    # Calculate the y gradient
    grad = -A @ (sigma_star_grad[:, np.newaxis] * softmin_grad) - L
    
    grad_flattened = grad.flatten()
    
    if np.isnan(grad_flattened).any() or np.isinf(grad_flattened).any():
        print("NaN or Inf detected in gradient!")
        return np.zeros_like(grad_flattened)
    
    return grad_flattened

def scaled_minimize(fun, x0, args=(), jac=None, **kwargs):
    # Determine the scale based on the maximum absolute value of initial guesses to prevent numerical issues
    scale = np.max(np.abs(x0)) + 1e-8
    
    # Define a scaled version of the original function to handle scale differences in variables
    def scaled_fun(x, *args):
        return fun(x * scale, *args)
    
    # If a Jacobian function is provided, scale it appropriately to correspond with the scaled function
    def scaled_jac(x, *args):
        return jac(x * scale, *args) * scale
    
    # Perform minimization on the scaled function and adjust the initial guess and Jacobian accordingly
    result = minimize(scaled_fun, x0 / scale, args=args, jac=scaled_jac if jac else None, **kwargs)
    
    # Adjust the result to reflect the original scale and return the scaled optimization result
    return OptimizeResult(
        x=result.x * scale,  # Scale the solution back to the original scale
        fun=result.fun,  # The function value at the solution
        jac=result.jac / scale if result.jac is not None else None,  # Scale the Jacobian back if it exists
        **{k: v for k, v in result.items() if k not in ['x', 'fun', 'jac']}  # Pass through other info unchanged
    )

In [ ]:
incidence_matrix = nx.incidence_matrix(graph, oriented=True).todense()
correspondence_matrix = np.array(correspondences.traffic_mat)
L = np.diag(correspondence_matrix.sum(axis=1)) - correspondence_matrix

print(L.shape)
print(incidence_matrix.shape)
n_ = incidence_matrix.shape[0]
m_ = incidence_matrix.shape[1]

# Initializing parameters for the method
y_init = np.zeros((n_, n_))
learning_rate = 0.01
num_iterations = 1000
fft, mu, rho, caps = get_graph_props(beckmann_model.graph)

mu = mu[0]
rho = rho[0]

-np.dot(incidence_matrix.T,y_init)[0]

s,e,te,fe,rho,mu=(-0.0, 0, 6.0, 25900.201171875, 0.15000000596046448, 0.25)

fe*(((s-te)/(te*rho))**mu)
((s-te)/(te*rho)) ** mu

incidence_matrix[:, 0]

x = np.arange(25).reshape((5,5)) 
# x[*[(0,1),(1,0)]] = np.array([100, 200])
x[np.arange(5), np.array([0, 1, 0, 1, 3])] = 1000
x

results = gradient_descent(y_init=np.zeros((n_, n_)), learning_rate=0.00001, num_iterations=100000, fw_results=flows_e_fw)
y_final, flow_values_avg, cons_log = results

y = results[0]

z = np.min(incidence_matrix.T @ y, axis=1)
amin = np.argmin(incidence_matrix.T @ y, axis=1)

flow_values = np.zeros((incidence_matrix.T.shape))
# amin_ind_tuples = np.vstack((np.arange(flow_values.shape[0]), amin)).T
# amin_ind_tuples
# flow_values[*amin_ind_tuples]
flow_values[np.arange(flow_values.shape[0]), amin] = 1
flow_values

cons_log = results[-1] 
plt.plot(cons_log)
plt.yscale("log")
plt.xlabel('Iterations')
plt.ylabel('Constraint violation')
plt.show()

# Create comparing GD & FW
plt.figure(figsize=(10, 6))
plt.plot(flows_e_fw, label="Frank-Wolfe", color='orange', marker='o')
plt.plot(flow_values_avg.sum(axis=1), label="Gradient Descent", color='blue', marker='x')
plt.xlabel('Edge index')
plt.ylabel('Flow on edge')
plt.title('Comparison of FW and GD Flow Values')
plt.legend()
plt.show()